**Regras de associação**

**Algoritmo Apriori**

**Veja: https://www.youtube.com/watch?v=0v6PiOpsJi8**

In [33]:
#!pip install apyori
#!pip install mlxtend

In [34]:
import pandas as pd
from apyori import apriori

In [35]:
#encoding = "cp1252", encoding='utf-8', encoding='latin1'etc
base= pd.read_csv('./PãoeManteiga SimNao.csv', sep=';',encoding='cp1252', header=0)
base

,Leite,Café,Cerveja,Pão,Manteiga,Arroz,Feijão
0,Não,Sim,Não,Sim,Sim,Não,Não
1,Sim,Não,Sim,Sim,Sim,Não,Não
2,Não,Sim,Não,Sim,Sim,Não,Não
3,Sim,Sim,Não,Sim,Sim,Não,Não
4,Não,Não,Sim,Não,Não,Não,Não
5,Não,Não,Não,Não,Sim,Não,Não
6,Não,Não,Não,Sim,Não,Não,Não
7,Não,Não,Não,Não,Não,Não,Sim
8,Não,Não,Não,Não,Não,Sim,Sim
9,Não,Não,Não,Não,Não,Sim,Não


In [36]:
base.shape

(10, 7)

**Como estaremos treinando um modelo a priori, que recebe entradas em formato de lista, precisamos transformar o dataframe em uma lista de transações.**

In [37]:
# 2. Renomear explicitamente as ausências
# Itera sobre todas as colunas do DataFrame
for col in base.columns:
    # Cria a nova string 'Não_NomeDoProduto'
    # Esta operação é crucial para transformar a AUSÊNCIA em um ITEM MINERÁVEL
    base[col] = base[col].apply(
        lambda x: col if x == 'Sim' else 'Não_' + col
    )

# 3. Gerar a Lista de Transações Corrigida
transacoes = []
# Percorre cada linha (transação)
for index, row in base.iterrows():
    # Adiciona TODOS os itens da linha, pois todos agora são strings únicas
    # (ou 'Produto' ou 'Não_Produto')
    transacoes.append(row.tolist())

# Exemplo: A transação agora é ['Não_Leite', 'Café', 'Não_Cerveja', ...]
print("Exemplo de transação para o Apriori (presença e ausência):")
print(transacoes[0])

Exemplo de transação para o Apriori (presença e ausência):
['Não_Leite', 'Café', 'Não_Cerveja', 'Pão', 'Manteiga', 'Não_Arroz', 'Não_Feijão']


In [38]:
transacoes

[['Não_Leite',
  'Café',
  'Não_Cerveja',
  'Pão',
  'Manteiga',
  'Não_Arroz',
  'Não_Feijão'],
 ['Leite',
  'Não_Café',
  'Cerveja',
  'Pão',
  'Manteiga',
  'Não_Arroz',
  'Não_Feijão'],
 ['Não_Leite',
  'Café',
  'Não_Cerveja',
  'Pão',
  'Manteiga',
  'Não_Arroz',
  'Não_Feijão'],
 ['Leite',
  'Café',
  'Não_Cerveja',
  'Pão',
  'Manteiga',
  'Não_Arroz',
  'Não_Feijão'],
 ['Não_Leite',
  'Não_Café',
  'Cerveja',
  'Não_Pão',
  'Não_Manteiga',
  'Não_Arroz',
  'Não_Feijão'],
 ['Não_Leite',
  'Não_Café',
  'Não_Cerveja',
  'Não_Pão',
  'Manteiga',
  'Não_Arroz',
  'Não_Feijão'],
 ['Não_Leite',
  'Não_Café',
  'Não_Cerveja',
  'Pão',
  'Não_Manteiga',
  'Não_Arroz',
  'Não_Feijão'],
 ['Não_Leite',
  'Não_Café',
  'Não_Cerveja',
  'Não_Pão',
  'Não_Manteiga',
  'Não_Arroz',
  'Feijão'],
 ['Não_Leite',
  'Não_Café',
  'Não_Cerveja',
  'Não_Pão',
  'Não_Manteiga',
  'Arroz',
  'Feijão'],
 ['Não_Leite',
  'Não_Café',
  'Não_Cerveja',
  'Não_Pão',
  'Não_Manteiga',
  'Arroz',
  'Não_Feij

In [39]:
type(transacoes)

list

**Vamos chamar executar o algoritmo apriori e armazenar as regras obtidas**

In [40]:
regras = apriori(transacoes, min_support = 0.3, min_confidence = 0.6, min_length = 1)
saida = list(regras)
print(len(saida))
print(saida)

136
[RelationRecord(items=frozenset({'Não_Arroz'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não_Arroz'}), confidence=0.8, lift=1.0)]), RelationRecord(items=frozenset({'Não_Café'}), support=0.7, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não_Café'}), confidence=0.7, lift=1.0)]), RelationRecord(items=frozenset({'Não_Cerveja'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não_Cerveja'}), confidence=0.8, lift=1.0)]), RelationRecord(items=frozenset({'Não_Feijão'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não_Feijão'}), confidence=0.8, lift=1.0)]), RelationRecord(items=frozenset({'Não_Leite'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não_Leite'}), confidence=0.8, lift=1.0)]), RelationRecord(items=frozenset({'Manteiga', 'Café'}), support

In [41]:
#Mostrar itemsets por tamanho
from collections import Counter
itemsets_por_tamanho = {}
min_support_percentual = 0.3
total_transacoes = len(transacoes)
min_support_count = min_support_percentual * total_transacoes

todos_os_itens = [item for sublist in transacoes for item in sublist]

contagem_itens = Counter(todos_os_itens)

itemsets_tamanho_1 = []
for item, contagem in contagem_itens.items():
    if contagem >= min_support_count:
        itemsets_tamanho_1.append({item})

print("\n--- Itemsets de Tamanho 1 ---")
print(f"Suporte Mínimo (Contagem): {min_support_count:.2f} (≈ {min_support_percentual*100}%)")
for itemset in itemsets_tamanho_1:
    item = list(itemset)[0]
    suporte = contagem_itens[item] / total_transacoes
    print(f"[{item}] - Suporte: {suporte:.3f} (Contagem: {contagem_itens[item]})")

for record in saida:
    itemset = set(record.items)
    tamanho_k = len(itemset)
    if tamanho_k not in itemsets_por_tamanho:
        itemsets_por_tamanho[tamanho_k] = []
        
    itemsets_por_tamanho[tamanho_k].append(itemset)

for k in sorted(itemsets_por_tamanho.keys()):
    print(f"\n--- Itemsets de Tamanho {k} ({len(itemsets_por_tamanho[k])} Total) ---")
    for itemset in itemsets_por_tamanho[k]:
        print(list(itemset))


--- Itemsets de Tamanho 1 ---
Suporte Mínimo (Contagem): 3.00 (≈ 30.0%)
[Não_Leite] - Suporte: 0.800 (Contagem: 8)
[Café] - Suporte: 0.300 (Contagem: 3)
[Não_Cerveja] - Suporte: 0.800 (Contagem: 8)
[Pão] - Suporte: 0.500 (Contagem: 5)
[Manteiga] - Suporte: 0.500 (Contagem: 5)
[Não_Arroz] - Suporte: 0.800 (Contagem: 8)
[Não_Feijão] - Suporte: 0.800 (Contagem: 8)
[Não_Café] - Suporte: 0.700 (Contagem: 7)
[Não_Pão] - Suporte: 0.500 (Contagem: 5)
[Não_Manteiga] - Suporte: 0.500 (Contagem: 5)

--- Itemsets de Tamanho 1 (5 Total) ---
['Não_Arroz']
['Não_Café']
['Não_Cerveja']
['Não_Feijão']
['Não_Leite']

--- Itemsets de Tamanho 2 (35 Total) ---
['Manteiga', 'Café']
['Café', 'Não_Arroz']
['Não_Cerveja', 'Café']
['Não_Feijão', 'Café']
['Pão', 'Café']
['Manteiga', 'Não_Arroz']
['Manteiga', 'Não_Cerveja']
['Não_Feijão', 'Manteiga']
['Não_Leite', 'Manteiga']
['Manteiga', 'Pão']
['Não_Café', 'Não_Arroz']
['Não_Cerveja', 'Não_Arroz']
['Não_Feijão', 'Não_Arroz']
['Não_Leite', 'Não_Arroz']
['Não_Ar

In [42]:
print(saida[0])


RelationRecord(items=frozenset({'Não_Arroz'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não_Arroz'}), confidence=0.8, lift=1.0)])


In [43]:
print(saida[1])

RelationRecord(items=frozenset({'Não_Café'}), support=0.7, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não_Café'}), confidence=0.7, lift=1.0)])


In [44]:
print(saida[2])

RelationRecord(items=frozenset({'Não_Cerveja'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não_Cerveja'}), confidence=0.8, lift=1.0)])


**Para facilitar a visualização, veja como transformar isso em um dataframe**

Extraído de:

https://www.section.io/engineering-education/apriori-algorithm-in-python/

https://splunktool.com/understanding-apyoris-output

In [45]:
Antecedente = []
Consequente = []
suporte = []
confianca = []
lift = []
RegrasFinais = []

for resultado in saida:
  s = resultado[1]
  result_rules = resultado[2]
  for result_rule in result_rules:
    a = list(result_rule[0])
    b = list(result_rule[1])
    c = result_rule[2]
    l = result_rule[3]
    if 'nan' in a or 'nan' in b: continue
    if len(a) == 0 or len(b) == 0: continue
    Antecedente.append(a)
    Consequente.append(b)
    suporte.append(s)
    confianca.append(c)
    lift.append(l)
    RegrasFinais = pd.DataFrame({'Antecedente': Antecedente, 'Consequente': Consequente, 'suporte': suporte, 'confianca': confianca, 'lift': lift})

In [46]:
RegrasFinais

,Antecedente,Consequente,suporte,confianca,lift
0,[Café],[Manteiga],0.3,1.0,2.000000
1,[Manteiga],[Café],0.3,0.6,2.000000
2,[Café],[Não_Arroz],0.3,1.0,1.250000
3,[Café],[Não_Cerveja],0.3,1.0,1.250000
4,[Café],[Não_Feijão],0.3,1.0,1.250000
...,...,...,...,...,...
874,"[Não_Cerveja, Não_Arroz, Pão, Café, Manteiga]",[Não_Feijão],0.3,1.0,1.250000
875,"[Não_Feijão, Não_Arroz, Pão, Café, Manteiga]",[Não_Cerveja],0.3,1.0,1.250000
876,"[Não_Feijão, Não_Cerveja, Pão, Café, Manteiga]",[Não_Arroz],0.3,1.0,1.250000
877,"[Não_Feijão, Não_Cerveja, Não_Arroz, Pão, Café]",[Manteiga],0.3,1.0,2.000000


**Podemos ordenar estas regras por uma métrica desejada**

In [47]:
RegrasFinais.sort_values(by='lift', ascending =False)

,Antecedente,Consequente,suporte,confianca,lift
823,[Café],"[Não_Feijão, Não_Cerveja, Não_Arroz, Pão, Mant...",0.3,1.0,3.333333
878,"[Não_Feijão, Não_Cerveja, Não_Arroz, Pão, Mant...",[Café],0.3,1.0,3.333333
827,"[Café, Não_Arroz]","[Não_Feijão, Manteiga, Pão, Não_Cerveja]",0.3,1.0,3.333333
829,"[Não_Feijão, Café]","[Manteiga, Pão, Não_Cerveja, Não_Arroz]",0.3,1.0,3.333333
633,"[Café, Não_Arroz]","[Manteiga, Pão, Não_Cerveja]",0.3,1.0,3.333333
...,...,...,...,...,...
483,"[Não_Leite, Não_Café, Não_Pão]",[Não_Arroz],0.3,0.6,0.750000
474,"[Não_Leite, Não_Café, Não_Manteiga]",[Não_Arroz],0.3,0.6,0.750000
515,"[Não_Feijão, Pão, Não_Arroz]",[Não_Leite],0.3,0.6,0.750000
24,[Não_Pão],[Não_Arroz],0.3,0.6,0.750000
